In [5]:
from Model import *
from View import *
from library import *

Scratchpad area: update the above segments as required with the model.py and library.py

In [6]:
video = Video("Tests/8/ball.mp4", (255, 255, 255))
print(video.getDimensions())
print(video.getFPS())

side = Video("Tests/8/side.mp4", (255, 255, 255))
print(side.getDimensions())
print(side.getFPS())

(1080, 1920)
119
(1920, 1080)
60


In [7]:
root = tk.Tk()
root.title("Backyard DRS")
video.incrementFrame()
view = VideoView(root, 540, 960)
view.getLabel().pack(side=tk.LEFT)
newFrame = video.getCurrentFrame()

rightFrame = tk.Frame(root)
rightFrame.pack(side=tk.LEFT, fill=tk.BOTH)
sideView = VideoView(rightFrame, 960, 540)
sideView.getLabel().pack(side=tk.TOP)
side.incrementFrame()
sideFrame = side.getCurrentFrame()
sideView.updateFrame(sideFrame)


def updateView(video: Video, view: VideoView):
    view.updateFrame(video.getCurrentFrame(), circles=video.getPoints(), cropRegion=video.getCropRegion())

def incrementAndUpdate(video: Video, view: VideoView):
    video.incrementFrame()
    updateView(video, view)

def updateParams(video: Video, view: VideoView, parameters):
    video.updateParameters(parameters)
    updateView(video, view)

def updateCropRegion(video: Video, view: VideoView, start, end):
    video.cropToRegion(start, end)
    updateView(video, view)

controlBar = VideoControlBar(
    rightFrame, "Front Angle", video.getDimensions(),
    lambda params: updateParams(video, view, params),
    lambda start, end: updateCropRegion(video, view, start, end),
    lambda: incrementAndUpdate(video, view),
    video.markFirstFrame
)
controlBar.getFrame().pack(side=tk.TOP, fill=tk.X)
controlBar2 = VideoControlBar(
    rightFrame, "Side Angle", side.getDimensions(),
    lambda params: updateParams(side, sideView, params),
    lambda start, end: updateCropRegion(side, sideView, start, end), 
    lambda: incrementAndUpdate(side, sideView),
    side.markFirstFrame
)
controlBar2.getFrame().pack(side=tk.TOP, fill=tk.X)

master = MasterControlBar(rightFrame, print, print)
master.getFrame().pack(side=tk.TOP, fill=tk.X)

root.mainloop() 

c:\Users\ayros\OneDrive\Documents\MENGINGEERING\Projects\Backyard DRS\BackyardDRS\library.py:52: RuntimeWarning: overflow encountered in scalar subtract
  dist = lambda x1,x2,y1,y2: (x1-x2)**2 + (y1-y2)**2


In [8]:
cv.destroyAllWindows()